
# content
1. Introduction
2. connecting to POSTGRES
3. creating the db schema
4. testing bitemporal crud


## 1. Introduction

If You start this repo in gitpod, You can open this notebook in vscode and execute the code with julia and postgres up and running
https://www.gitpod.io/docs/

### 1.1. connecting to POSTGRES

In [4]:
include("init.jl")


  Activating project at `/workspaces/BitemporalPostgres.jl`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.toml`
  No Changes to `/workspaces/BitemporalPostgres.jl/Manifest.toml`
   Resolving package versions...
  No Changes to `/workspaces/BitemporalPostgres.jl/Project.t

In [5]:
using SearchLight
using SearchLightPostgreSQL
SearchLight.connect(SearchLight.Configuration.load())
SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
SearchLight.Migrations.create_migrations_table()
BitemporalPostgres.up()

┌ Info: DROP SCHEMA public CASCADE
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  drop cascades to 13 other objects
DETAIL:  drop cascades to table schema_migrations
drop cascades to table histories
drop cascades to table versions
drop cascades to table workflows
drop cascades to table testtstzranges
drop cascades to table validityintervals
drop cascades to table testdummycomponents
drop cascades to table testdummycomponentrevisions
drop cascades to table testdummysubcomponents
drop cascades to table testdummysubcomponentrevisions
drop cascades to extension btree_gist
drop cascades to function f_bitempranges()
drop cascades to function f_versionrange()
┌ Info: CREATE SCHEMA public
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/codespace/.julia/

0×0 DataFrame

# 2 Starting with workflows

## 2.1 Workflow 1 (the blue rectangle :=) )

### 2.1.1 Starting workflow 1
inserting component 1 and subcomponent 1.1

In [6]:
using Dates, Test, TimeZones
import BitemporalPostgres
using BitemporalPostgres

w1blue = Workflow(type_of_entity="TestDummyComponent",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"))

t1 = TestDummyComponent()
t1r1blue = TestDummyComponentRevision(description="blue")
ts = TestDummySubComponent(ref_super=t1.id)
ts1r1green = TestDummySubComponentRevision(description="green")
create_entity!(w1blue)
create_component!(t1, t1r1blue, w1blue)
println(t1r1blue)
create_subcomponent!(t1, ts, ts1r1green, w1blue)
@test !isnothing(w1blue.ref_history)
@test w1blue.is_committed == 0
@test w1blue.ref_version == t1r1blue.ref_validfrom
@test w1blue.ref_version == ts1r1green.ref_validfrom

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/Se

TestDummyComponentRevision(ref_validfrom::DbId=1,ref_invalidfrom::DbId=9007199254740991,id::DbId=1,ref_component::DbId=1,description::String=blue)


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO testdummysubcomponents ( "ref_history", "ref_super", "ref_version" ) VALUES ( E'1', E'1', E'9007199254740991' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummysubcomponents"."id" AS "testdummysubcomponents_id", "testdummysubcomponents"."ref_history" AS "testdummysubcomponents_ref_history", "testdummysubcomponents"."ref_super" AS "testdummysubcomponents_ref_super", "testdummysubcomponents"."ref_version" AS "testdummysubcomponents_ref_version" FROM "testdummysubcomponents" WHERE "id" = 1 ORDER BY testdummysubcomponents.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPost

Test Passed

### 2.1.2 Commiting workflow 1

In [7]:
commit_workflow!(w1blue)
@test w1blue.is_committed == 1

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'1' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" A

Test Passed

## 2.2 Workflow 2 ( the yellow rectangle that shortens the blue one)
### 2.2.1  Starting workflow 2 
mutating component 1

In [8]:
w2yellow = Workflow(type_of_entity="TestDummyComponent",
    ref_history=w1blue.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
t1r2yellow = copy(t1r1blue)
t1r2yellow.description = "yellow"
update_entity!(w2yellow)
update_component!(t1r1blue, t1r2yellow, w2yellow)
println(t1r2yellow)
@test w2yellow.ref_version == t1r2yellow.ref_validfrom
@test w2yellow.ref_version == t1r1blue.ref_invalidfrom

┌ Info: entering update
└ @ BitemporalPostgres /workspaces/BitemporalPostgres.jl/src/BitemporalPostgres.jl:622
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 2 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'1', E'2', E'2024-06-04T09:10:32.904+00:00', E'2015-05-30T21:00:01.001+00:00', 0, E'TestDummyComponent' ) RETURNING id
└ @ SearchLightPostgreSQL

TestDummyComponentRevision(ref_validfrom::DbId=2,ref_invalidfrom::DbId=9007199254740991,id::DbId=2,ref_component::DbId=1,description::String=yellow)


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE testdummycomponentrevisions SET  "id" = E'1', "ref_component" = E'1', "ref_validfrom" = E'1', "ref_invalidfrom" = E'2', "description" = E'blue' WHERE testdummycomponentrevisions.id = '1' RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (1,1,1,2,"[1,9007199254740991)",blue)
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentre

Test Passed

### 2.2.2 Committing workflow 2

In [9]:
commit_workflow!(w2yellow)
@test w2yellow.is_committed == 1

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'2' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" A

Test Passed

## 2.3 workflow 3 (the red rectancle that shadows the yellow one)
### 2.3.1 Starting workflow 3

In [10]:
w3redshadow = Workflow(type_of_entity="TestDummyComponent",
    ref_history = w1blue.ref_history,
    tsw_validfrom = ZonedDateTime(2014, 11, 30, 21, 0, 1, 1, tz"UTC"),
)

Workflow
| KEY                           | VALUE                         |
|-------------------------------|-------------------------------|
| id::DbId                      | NULL                          |
| is_committed::Int64           | 0                             |
| ref_history::DbId             | 1                             |
| ref_version::DbId             | 0                             |
| tsdb_validfrom::ZonedDateTime | 2038-01-19T03:14:06.999+00:00 |
| tsw_validfrom::ZonedDateTime  | 2014-11-30T21:00:01.001+00:00 |
| type_of_entity::String        | TestDummyComponent            |


In [11]:
update_entity!(w3redshadow)
t1r1blue = findcomponentrevision(TestDummyComponentRevision, t1.id, w3redshadow.ref_version)[1]
t1r3red = copy(t1r1blue)
t1r3red.description = "red"
t1r3red

┌ Info: entering update
└ @ BitemporalPostgres /workspaces/BitemporalPostgres.jl/src/BitemporalPostgres.jl:622
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 3 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'1', E'3', E'2024-06-04T09:10:34.544+00:00', E'2014-11-30T21:00:01.001+00:00', 0, E'TestDummyComponent' ) RETURNING id
└ @ SearchLightPostgreSQL

TestDummyComponentRevision
| KEY                   | VALUE            |
|-----------------------|------------------|
| description::String   | red              |
| id::DbId              | NULL             |
| ref_component::DbId   | 1                |
| ref_invalidfrom::DbId | 9007199254740991 |
| ref_validfrom::DbId   | 9007199254740991 |


In [12]:
update_component!(t1r1blue, t1r3red, w3redshadow)
@test w3redshadow.ref_version == t1r3red.ref_validfrom


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE testdummycomponentrevisions SET  "id" = E'3', "ref_component" = E'1', "ref_validfrom" = E'3', "ref_invalidfrom" = E'3', "description" = E'blue' WHERE testdummycomponentrevisions.id = '3' RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (3,1,3,3,"[3,9007199254740991)",blue)
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentrevisions_ref_invalidfrom", "testdummycomponentrevisions"."description" AS "testdummycomponentrevisions_description" FROM "testdummycomp

Test Passed

### 2.3.4 Committing workflow 3

In [13]:
commit_workflow!(w3redshadow)
@test w3redshadow.is_committed == 1

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'3' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" A

Test Passed

# 3 Testing 
For each workflow 
* as of its intervals valid from points
* as of its intervals db valid from and 1 second before its world validfrom

In [14]:
v1 = findversion(w1blue.ref_history, w1blue.tsdb_validfrom, w1blue.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t1.id, v1)[1].description == "blue"

v2 = findversion(w2yellow.ref_history, w2yellow.tsdb_validfrom, w2yellow.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t1.id, v2)[1].description == "yellow"

v2a = findversion(w2yellow.ref_history, w2yellow.tsdb_validfrom, w2yellow.tsw_validfrom - Dates.Second(1))

@test findcomponentrevision(TestDummyComponentRevision, t1.id, v2a)[1].description == "blue"

v3 = findversion(w3redshadow.ref_history, w3redshadow.tsdb_validfrom, w3redshadow.tsw_validfrom)

@test findcomponentrevision(TestDummyComponentRevision, t1.id, v3)[1].description == "red"

v3a = findversion(w3redshadow.ref_history, w3redshadow.tsdb_validfrom, w3redshadow.tsw_validfrom - Dates.Second(1))

@test findcomponentrevision(TestDummyComponentRevision, t1.id, v3a)[1].description == "blue"


┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history=E'1' and tsrworld @> TIMESTAMPTZ E'2014-05-30T21:00:01.001+00:00' AND tsrdb @> TIMESTAMPTZ E'2024-06-04T09:10:32.384+00:00' AND is_committed = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "tes

Test Passed

In [15]:
v1 = findversion(w1blue.ref_history, w1blue.tsdb_validfrom, w1blue.tsw_validfrom)
r1=findcomponentrevision(TestDummySubComponentRevision, ts.id, v1)
println(v1)

println(r1)

1


┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history=E'1' and tsrworld @> TIMESTAMPTZ E'2014-05-30T21:00:01.001+00:00' AND tsrdb @> TIMESTAMPTZ E'2024-06-04T09:10:32.384+00:00' AND is_committed = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummysubcomponentrevisions"."id" AS "

TestDummySubComponentRevision[TestDummySubComponentRevision
| KEY                   | VALUE            |
|-----------------------|------------------|
| description::String   | green            |
| id::DbId              | 1                |
| ref_component::DbId   | 1                |
| ref_invalidfrom::DbId | 9007199254740991 |
| ref_validfrom::DbId   | 1                |
]


In [16]:
@testset "reading tests" begin
    v1 = findversion(w1blue.ref_history, w1blue.tsdb_validfrom, w1blue.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v1)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v1)[1].description == "green"
    
    v2 = findversion(w2yellow.ref_history, w2yellow.tsdb_validfrom, w2yellow.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v2)[1].description == "yellow"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v2)[1].description == "green"
    
    v2a = findversion(w2yellow.ref_history, w2yellow.tsdb_validfrom, w2yellow.tsw_validfrom - Dates.Second(1))
    
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v2a)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v2a)[1].description == "green"
    
    
    v3 = findversion(w3redshadow.ref_history, w3redshadow.tsdb_validfrom, w3redshadow.tsw_validfrom)
    
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v3)[1].description == "red"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v3)[1].description == "green"
    
    v3a = findversion(w3redshadow.ref_history, w3redshadow.tsdb_validfrom, w3redshadow.tsw_validfrom - Dates.Second(1))
    
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, v3a)[1].description == "blue"
    @test findcomponentrevision(TestDummySubComponentRevision, ts.id, v3a)[1].description == "green"
    
    end

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history=E'1' and tsrworld @> TIMESTAMPTZ E'2014-05-30T21:00:01.001+00:00' AND tsrdb @> TIMESTAMPTZ E'2024-06-04T09:10:32.384+00:00' AND is_committed = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "tes

Test Summary: | Pass  Total  Time
reading tests |   10     10  0.1s


Test.DefaultTestSet("reading tests", Any[], 10, false, false, true, 1.717492235712023e9, 1.717492235840191e9, false, "/workspaces/BitemporalPostgres.jl/bitemporal_testcase.ipynb")

Testing pending transactions and rollback

In [17]:
w4PendingRollback = Workflow(type_of_entity="TestDummyComponent",
    ref_history=w1blue.ref_history,
    tsdb_validfrom=now(tz"UTC"),
    tsw_validfrom=ZonedDateTime(2017, 11, 30, 21, 0, 1, 1, tz"UTC"),
)
update_entity!(w4PendingRollback)

t1r3red = findcomponentrevision(TestDummyComponentRevision, t1.id, w4PendingRollback.ref_version)[1]
t1r4green = copy(t1r3red)
t1r4green.description = "green"

t2 = TestDummyComponent()
t2r1pink = TestDummyComponentRevision(description="pink")

@test t1r3red.ref_invalidfrom == MaxVersion
update_component!(t1r3red, t1r4green, w4PendingRollback)
create_component!(t2, t2r1pink, w4PendingRollback)


┌ Info: entering update
└ @ BitemporalPostgres /workspaces/BitemporalPostgres.jl/src/BitemporalPostgres.jl:622
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 4 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'1', E'4', E'2024-06-04T09:10:36.564+00:00', E'2017-11-30T21:00:01.001+00:00', 0, E'TestDummyComponent' ) RETURNING id
└ @ SearchLightPostgreSQL

In [18]:
@testset "pending transaction tests" begin
    @test findcomponentrevision(TestDummyComponentRevision, t1.id, w4PendingRollback.ref_version)[1].description == "green"
    @test findcomponentrevision(TestDummyComponentRevision, t2.id, w4PendingRollback.ref_version)[1].description == "pink"
    @test w4PendingRollback.ref_version == t1r3red.ref_invalidfrom
    @test w4PendingRollback.ref_version == t1r4green.ref_validfrom
    @test MaxVersion == t1r4green.ref_invalidfrom
    @test w4PendingRollback.ref_version == t2r1pink.ref_validfrom
end


Test Summary:             | Pass  Total  Time
pending transaction tests |    6      6  0.0s


┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentrevisions_ref_invalidfrom", "testdummycomponentrevisions"."description" AS "testdummycomponentrevisions_description" FROM "testdummycomponentrevisions" WHERE ref_component = E'1' AND ref_valid @> BIGINT E'4' ORDER BY testdummycomponentrevisions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdumm

Test.DefaultTestSet("pending transaction tests", Any[], 6, false, false, true, 1.717492236606955e9, 1.717492236608172e9, false, "/workspaces/BitemporalPostgres.jl/bitemporal_testcase.ipynb")

In [19]:
@testset "rollbacked transaction tests" begin
    rollback_workflow!(w4PendingRollback)
    @test !isempty(findcomponentrevision(TestDummyComponentRevision, t1.id, w4PendingRollback.ref_version))
    w4PendingRollback.ref_version == t1r3red.ref_invalidfrom
    w4PendingRollback.ref_version == t1r4green.ref_validfrom
end

Test Summary:                | Pass  Total  Time
rollbacked transaction tests |    1      1  0.1s


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE id=E'4' ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: DELETE FROM versions WHERE id = '4'
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (4,1,3,9007199254740991,"[3,4)",red)
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycompo

Test.DefaultTestSet("rollbacked transaction tests", Any[], 1, false, false, true, 1.717492236615547e9, 1.717492236725449e9, false, "/workspaces/BitemporalPostgres.jl/bitemporal_testcase.ipynb")

In [20]:
w3redshadow = Workflow(type_of_entity="TestDummyComponent",
    ref_history = w1blue.ref_history,
    tsw_validfrom = ZonedDateTime(2014, 11, 30, 21, 0, 1, 1, tz"UTC"),
)

Workflow
| KEY                           | VALUE                         |
|-------------------------------|-------------------------------|
| id::DbId                      | NULL                          |
| is_committed::Int64           | 0                             |
| ref_history::DbId             | 1                             |
| ref_version::DbId             | 0                             |
| tsdb_validfrom::ZonedDateTime | 2038-01-19T03:14:06.999+00:00 |
| tsw_validfrom::ZonedDateTime  | 2014-11-30T21:00:01.001+00:00 |
| type_of_entity::String        | TestDummyComponent            |


### 2.3.2 Testing retrospective transactions
#### 2.3.2.1 preparing the retrospective transaction by 
* preliminarily invalidating all insertions and mutations from shadowed versions and 
* reviving all revisions invalidated by shadowed versions

In [21]:
update_entity!(w3redshadow)
@testset "retrospective update entity red revive and invalidate shadowed " begin
    # have currently shadowed revisions been invalidated?
    @test !isempty(find(TestDummyComponentRevision, SQLWhereExpression("ref_invalidfrom=?", w3redshadow.ref_version)))
    # have revisions invalidated by shadowed versions been revived?
    @test !isempty(find(TestDummyComponentRevision, SQLWhereExpression("ref_validfrom=?", w3redshadow.ref_version)))
end


┌ Info: entering update
└ @ BitemporalPostgres /workspaces/BitemporalPostgres.jl/src/BitemporalPostgres.jl:622
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 5 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'1', E'5', E'2024-06-04T09:10:36.735+00:00', E'2014-11-30T21:00:01.001+00:00', 0, E'TestDummyComponent' ) RETURNING id
└ @ SearchLightPostgreSQL

Test Summary:                                                   | Pass  Total  Time
retrospective update entity red revive and invalidate shadowed  |    2      2  0.0s


┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentrevisions_ref_invalidfrom", "testdummycomponentrevisions"."description" AS "testdummycomponentrevisions_description" FROM "testdummycomponentrevisions" WHERE ref_validfrom=E'5' ORDER BY testdummycomponentrevisions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


Test.DefaultTestSet("retrospective update entity red revive and invalidate shadowed ", Any[], 2, false, false, true, 1.717492236753396e9, 1.717492236761658e9, false, "/workspaces/BitemporalPostgres.jl/bitemporal_testcase.ipynb")

#### 2.3.2.2 Does rolling back the transaction delete the preliminary revisions? 

In [22]:
 
rollback_workflow!(w3redshadow)
@testset "retrospective update entity red revive and invalidate shadowed " begin
    # have currently shadowed revisions been invalidated?
    @test isempty(find(TestDummyComponentRevision, SQLWhereExpression("ref_invalidfrom=?", w3redshadow.ref_version)))
    # have revisions invalidated by shadowed versions been revived?
    @test isempty(find(TestDummyComponentRevision, SQLWhereExpression("ref_validfrom=?", w3redshadow.ref_version)))
end


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE id=E'5' ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: DELETE FROM versions WHERE id = '5'
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (4,1,3,9007199254740991,"[3,5)",red)
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


Test Summary:                                                   | Pass  Total  Time
retrospective update entity red revive and invalidate shadowed  |    2      2  0.0s


┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_invalidfrom" AS "testdummycomponentrevisions_ref_invalidfrom", "testdummycomponentrevisions"."description" AS "testdummycomponentrevisions_description" FROM "testdummycomponentrevisions" WHERE ref_invalidfrom=E'5' ORDER BY testdummycomponentrevisions.id ASC
└ @ SearchLightPostgreSQL /home/codespace/.julia/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "testdummycomponentrevisions"."id" AS "testdummycomponentrevisions_id", "testdummycomponentrevisions"."ref_component" AS "testdummycomponentrevisions_ref_component", "testdummycomponentrevisions"."ref_validfrom" AS "testdummycomponentrevisions_ref_validfrom", "testdummycomponentrevisions"."ref_inv

Test.DefaultTestSet("retrospective update entity red revive and invalidate shadowed ", Any[], 2, false, false, true, 1.717492236778211e9, 1.717492236779186e9, false, "/workspaces/BitemporalPostgres.jl/bitemporal_testcase.ipynb")